In [2]:
#导入必要的库
%matplotlib inline

#==================自带库===================
from os import listdir,makedirs
from os.path import join,exists,expanduser

#====================数据处理库==============
import numpy as np
import pandas as pd

#===================绘图库==================
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

#===================时间,进度处理============
from tqdm import tqdm
import datetime as dt

#================= sklearn=================
from sklearn.metrics import log_loss,accuracy_score
from sklearn.linear_model import LogisticRegression

#====================Keras=================
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input,decode_predictions


print("import done")

import done


In [18]:
INPUT_SIZE = 224
NUM_CLASSES = 16
SEED = 1987
data_dir = 'data/train'
labels = pd.read_csv('data/labels.csv')
#print(len(listdir(data_dir)),len(labels))
# 10222 10222
selected_breed_list = list(labels.groupby('breed').count().sort_values(
    by='id',ascending=False).head(NUM_CLASSES).index)
# ['scottish_deerhound', 'maltese_dog', 'afghan_hound', 'entlebucher', 
#  'bernese_mountain_dog', 'shih-tzu', 'great_pyrenees', 'pomeranian',
#  'basenji', 'samoyed', 'airedale', 'tibetan_terrier', 'leonberg', 
#  'cairn', 'beagle', 'japanese_spaniel']
labels = labels[labels['breed'].isin(selected_breed_list)]
# [1777 rows x 2 columns]

labels['target'] = 1
labels['rank'] = labels.groupby('breed').rank()['id']
labels_pivot = labels.pivot('id','breed','target').reset_index().fillna(0)

np.random.seed(seed=SEED)
rnd = np.random.random(len(labels))
train_idx = rnd < 0.8
valid_idx = rnd >= 0.8

y_train = labels_pivot[selected_breed_list].values
ytr = y_train[train_idx]
yv = y_train[valid_idx]


In [ ]:
def read_img(img_id, train_or_test, size):
    """Read and resize image
       # Arguments
           img_id:string
           train_or_test:string 'train' or 'test'
           size:resize the original image
           
        # Returns
           Image as numpy array
    """
     img = image.load_img()